In [2]:
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import rgb2hex
from descartes import PolygonPatch
from shapely.geometry import Polygon, MultiPolygon

In [3]:
import pandas as pd 
df = pd.read_json('./geotagged_tweets_20160812-0912.jsons', lines=True)

In [4]:
df.shape

(657307, 30)

In [7]:
df.columns

Index(['created_at', 'id', 'id_str', 'text', 'source', 'truncated',
       'in_reply_to_status_id', 'in_reply_to_status_id_str',
       'in_reply_to_user_id', 'in_reply_to_user_id_str',
       'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place',
       'contributors', 'is_quote_status', 'retweet_count', 'favorite_count',
       'entities', 'extended_entities', 'favorited', 'retweeted',
       'possibly_sensitive', 'filter_level', 'lang', 'timestamp_ms',
       'quoted_status_id', 'quoted_status_id_str', 'quoted_status'],
      dtype='object')

Selecting only columns that we decided to keep:
- created_at
- text
- user.id
- user.name
- user.followers_count
- user.friends_count
- place.country
- place.full_name
- retweet_count
- favorite_count
- lang

In [100]:
reduced_df = df[["created_at", "text", "user", "place", "lang", 'retweet_count', 'favorite_count']]
reduced_df.head()

,created_at,text,user,place,lang,retweet_count,favorite_count
0,2016-08-12 10:04:00+00:00,@theblaze @realDonaldTrump https://t.co/TY9DlZ...,"{'id': 366636488, 'id_str': '366636488', 'name...","{'id': '29a119f18820c3ad', 'url': 'https://api...",und,0,0
1,2016-08-12 10:04:02+00:00,@BarackObama \n@FBI\n@LORETTALYNCH \nALL IN CO...,"{'id': 82496193, 'id_str': '82496193', 'name':...","{'id': 'c7ef5f3368b68777', 'url': 'https://api...",en,0,0
2,2016-08-12 10:04:10+00:00,@theblaze @realDonaldTrump https://t.co/n050DB...,"{'id': 366636488, 'id_str': '366636488', 'name...","{'id': '29a119f18820c3ad', 'url': 'https://api...",und,0,0
3,2016-08-12 10:04:21+00:00,@HillaryClinton he will do in one year all the...,"{'id': 44032624, 'id_str': '44032624', 'name':...","{'id': '01864a8a64df9dc4', 'url': 'https://api...",en,0,0
4,2016-08-12 10:04:30+00:00,#CNN #newday clear #Trump deliberately throwin...,"{'id': 769208504, 'id_str': '769208504', 'name...","{'id': 'c0b8e8dc81930292', 'url': 'https://api...",en,0,0


As we see we have two dictionaries within the columns we have selected and now we want to unnest those dictionaries and concatenate them with our original table.

In [127]:
reduced_with_place = pd.concat([reduced_df.drop(['place'], axis=1), reduced_df['place'].apply(pd.Series)], axis=1)
reduced_with_place

,created_at,text,user,lang,retweet_count,favorite_count,id,url,place_type,name,full_name,country_code,country,bounding_box,attributes
0,2016-08-12 10:04:00+00:00,@theblaze @realDonaldTrump https://t.co/TY9DlZ...,"{'id': 366636488, 'id_str': '366636488', 'name...",und,0,0,29a119f18820c3ad,https://api.twitter.com/1.1/geo/id/29a119f1882...,city,Frontenac,"Frontenac, MO",US,United States,"{'type': 'Polygon', 'coordinates': [[[-90.4339...",{}
1,2016-08-12 10:04:02+00:00,@BarackObama \n@FBI\n@LORETTALYNCH \nALL IN CO...,"{'id': 82496193, 'id_str': '82496193', 'name':...",en,0,0,c7ef5f3368b68777,https://api.twitter.com/1.1/geo/id/c7ef5f3368b...,city,Baton Rouge,"Baton Rouge, LA",US,United States,"{'type': 'Polygon', 'coordinates': [[[-91.2189...",{}
2,2016-08-12 10:04:10+00:00,@theblaze @realDonaldTrump https://t.co/n050DB...,"{'id': 366636488, 'id_str': '366636488', 'name...",und,0,0,29a119f18820c3ad,https://api.twitter.com/1.1/geo/id/29a119f1882...,city,Frontenac,"Frontenac, MO",US,United States,"{'type': 'Polygon', 'coordinates': [[[-90.4339...",{}
3,2016-08-12 10:04:21+00:00,@HillaryClinton he will do in one year all the...,"{'id': 44032624, 'id_str': '44032624', 'name':...",en,0,0,01864a8a64df9dc4,https://api.twitter.com/1.1/geo/id/01864a8a64d...,city,Melbourne,"Melbourne, Victoria",AU,Australia,"{'type': 'Polygon', 'coordinates': [[[144.5937...",{}
4,2016-08-12 10:04:30+00:00,#CNN #newday clear #Trump deliberately throwin...,"{'id': 769208504, 'id_str': '769208504', 'name...",en,0,0,c0b8e8dc81930292,https://api.twitter.com/1.1/geo/id/c0b8e8dc819...,city,Baltimore,"Baltimore, MD",US,United States,"{'type': 'Polygon', 'coordinates': [[[-76.7115...",{}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
657302,2016-09-12 13:20:32+00:00,"TRUMP U, TAXES ,WEIRD MEDICAL REPORT WITH A WH...","{'id': 569487350, 'id_str': '569487350', 'name...",en,0,0,a592bd6ceb1319f7,https://api.twitter.com/1.1/geo/id/a592bd6ceb1...,city,San Diego,"San Diego, CA",US,United States,"{'type': 'Polygon', 'coordinates': [[[-117.282...",{}
657303,2016-09-12 13:20:33+00:00,@CarolCNN if MSM were honest watch any utube v...,"{'id': 716017946166857728, 'id_str': '71601794...",en,0,0,778af41ffb719450,https://api.twitter.com/1.1/geo/id/778af41ffb7...,city,Coral Gables,"Coral Gables, FL",US,United States,"{'type': 'Polygon', 'coordinates': [[[-80.2971...",{}
657304,2016-09-12 13:20:36+00:00,#EEUU\nNeumonía obliga a @HillaryClinton a can...,"{'id': 44489439, 'id_str': '44489439', 'name':...",es,0,0,c4320f726d222937,https://api.twitter.com/1.1/geo/id/c4320f726d2...,country,Bolivia,Bolivia,BO,Bolivia,"{'type': 'Polygon', 'coordinates': [[[-69.6407...",{}
657305,2016-09-12 13:20:38+00:00,It's interesting that Hillary Clinton's crowds...,"{'id': 3241116564, 'id_str': '3241116564', 'na...",en,0,0,174af231a0d9f46c,https://api.twitter.com/1.1/geo/id/174af231a0d...,city,Fairbanks,"Fairbanks, AK",US,United States,"{'type': 'Polygon', 'coordinates': [[[-147.813...",{}


Now we want to select only relevant columns from the unnested dataframe and then in order to know whats inside of them we need to rename them.

In [128]:
reduced_with_place = reduced_with_place[["created_at", "text", "user", "full_name","country", "lang", 'retweet_count', 'favorite_count']]
reduced_with_place = reduced_with_place.rename(columns = {"full_name":"place_full_name"})
reduced_with_place

,created_at,text,user,place_full_name,country,lang,retweet_count,favorite_count
0,2016-08-12 10:04:00+00:00,@theblaze @realDonaldTrump https://t.co/TY9DlZ...,"{'id': 366636488, 'id_str': '366636488', 'name...","Frontenac, MO",United States,und,0,0
1,2016-08-12 10:04:02+00:00,@BarackObama \n@FBI\n@LORETTALYNCH \nALL IN CO...,"{'id': 82496193, 'id_str': '82496193', 'name':...","Baton Rouge, LA",United States,en,0,0
2,2016-08-12 10:04:10+00:00,@theblaze @realDonaldTrump https://t.co/n050DB...,"{'id': 366636488, 'id_str': '366636488', 'name...","Frontenac, MO",United States,und,0,0
3,2016-08-12 10:04:21+00:00,@HillaryClinton he will do in one year all the...,"{'id': 44032624, 'id_str': '44032624', 'name':...","Melbourne, Victoria",Australia,en,0,0
4,2016-08-12 10:04:30+00:00,#CNN #newday clear #Trump deliberately throwin...,"{'id': 769208504, 'id_str': '769208504', 'name...","Baltimore, MD",United States,en,0,0
...,...,...,...,...,...,...,...,...
657302,2016-09-12 13:20:32+00:00,"TRUMP U, TAXES ,WEIRD MEDICAL REPORT WITH A WH...","{'id': 569487350, 'id_str': '569487350', 'name...","San Diego, CA",United States,en,0,0
657303,2016-09-12 13:20:33+00:00,@CarolCNN if MSM were honest watch any utube v...,"{'id': 716017946166857728, 'id_str': '71601794...","Coral Gables, FL",United States,en,0,0
657304,2016-09-12 13:20:36+00:00,#EEUU\nNeumonía obliga a @HillaryClinton a can...,"{'id': 44489439, 'id_str': '44489439', 'name':...",Bolivia,Bolivia,es,0,0
657305,2016-09-12 13:20:38+00:00,It's interesting that Hillary Clinton's crowds...,"{'id': 3241116564, 'id_str': '3241116564', 'na...","Fairbanks, AK",United States,en,0,0


Below we do the same thing for the user column:

In [129]:
reduced_with_place_and_user = pd.concat([reduced_with_place.drop(['user'], axis=1), reduced_with_place['user'].apply(pd.Series)], axis=1)
reduced_with_place_and_user = reduced_with_place_and_user[["created_at", "text", "id","name", "place_full_name","country", "lang", 'retweet_count', 'favorite_count']]
reduced_with_place_and_user = reduced_with_place_and_user.rename(columns = {"id":"user_id","name":"user_name"})
reduced_with_place_and_user.head()

,created_at,created_at,text,user_id,user_name,place_full_name,country,lang,lang,retweet_count,favorite_count
0,2016-08-12 10:04:00+00:00,Fri Sep 02 14:54:17 +0000 2011,@theblaze @realDonaldTrump https://t.co/TY9DlZ...,366636488,GIL DUPUY,"Frontenac, MO",United States,und,en,0,0
1,2016-08-12 10:04:02+00:00,Thu Oct 15 00:28:04 +0000 2009,@BarackObama \n@FBI\n@LORETTALYNCH \nALL IN CO...,82496193,Red Octopus,"Baton Rouge, LA",United States,en,en,0,0
2,2016-08-12 10:04:10+00:00,Fri Sep 02 14:54:17 +0000 2011,@theblaze @realDonaldTrump https://t.co/n050DB...,366636488,GIL DUPUY,"Frontenac, MO",United States,und,en,0,0
3,2016-08-12 10:04:21+00:00,Tue Jun 02 01:59:59 +0000 2009,@HillaryClinton he will do in one year all the...,44032624,JanJorgenson,"Melbourne, Victoria",Australia,en,en,0,0
4,2016-08-12 10:04:30+00:00,Mon Aug 20 09:43:48 +0000 2012,#CNN #newday clear #Trump deliberately throwin...,769208504,Beverly Spence,"Baltimore, MD",United States,en,en,0,0
...,...,...,...,...,...,...,...,...,...,...,...
657302,2016-09-12 13:20:32+00:00,Wed May 02 22:48:50 +0000 2012,"TRUMP U, TAXES ,WEIRD MEDICAL REPORT WITH A WH...",569487350,Sheryl Berghoff,"San Diego, CA",United States,en,en,0,0
657303,2016-09-12 13:20:33+00:00,Fri Apr 01 21:42:36 +0000 2016,@CarolCNN if MSM were honest watch any utube v...,716017946166857728,Karen B,"Coral Gables, FL",United States,en,en,0,0
657304,2016-09-12 13:20:36+00:00,Wed Jun 03 23:34:49 +0000 2009,#EEUU\nNeumonía obliga a @HillaryClinton a can...,44489439,La Razon Digital,Bolivia,Bolivia,es,es,0,0
657305,2016-09-12 13:20:38+00:00,Wed Jun 10 08:43:55 +0000 2015,It's interesting that Hillary Clinton's crowds...,3241116564,Robert Chaffin,"Fairbanks, AK",United States,en,en,0,0


But as we see we have duplicated column names, "created_at" and "lang" and we need to manually change their names for being able to distinct them.

In [136]:
reduced_with_place_and_user.columns = ["tweet_created_at", "user_created_at","text", "user_id","user_name", "place_full_name","country", "tweet_lang", "user_lang", 'retweet_count', 'favorite_count']
reduced_with_place_and_user.head()

,tweet_created_at,user_created_at,text,user_id,user_name,place_full_name,country,tweet_lang,user_lang,retweet_count,favorite_count
0,2016-08-12 10:04:00+00:00,Fri Sep 02 14:54:17 +0000 2011,@theblaze @realDonaldTrump https://t.co/TY9DlZ...,366636488,GIL DUPUY,"Frontenac, MO",United States,und,en,0,0
1,2016-08-12 10:04:02+00:00,Thu Oct 15 00:28:04 +0000 2009,@BarackObama \n@FBI\n@LORETTALYNCH \nALL IN CO...,82496193,Red Octopus,"Baton Rouge, LA",United States,en,en,0,0
2,2016-08-12 10:04:10+00:00,Fri Sep 02 14:54:17 +0000 2011,@theblaze @realDonaldTrump https://t.co/n050DB...,366636488,GIL DUPUY,"Frontenac, MO",United States,und,en,0,0
3,2016-08-12 10:04:21+00:00,Tue Jun 02 01:59:59 +0000 2009,@HillaryClinton he will do in one year all the...,44032624,JanJorgenson,"Melbourne, Victoria",Australia,en,en,0,0
4,2016-08-12 10:04:30+00:00,Mon Aug 20 09:43:48 +0000 2012,#CNN #newday clear #Trump deliberately throwin...,769208504,Beverly Spence,"Baltimore, MD",United States,en,en,0,0


In [137]:
reduced_with_place_and_user = reduced_with_place_and_user[reduced_with_place_and_user.tweet_lang == "en"]
reduced_with_place_and_user.to_csv("reduced_tweets_only_eng.csv")